<a href="https://colab.research.google.com/github/pra17dod/Waste-Segregation/blob/main/model/v4_waste_segregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import re
import random
import seaborn as sns
import cv2 as cv
from PIL import Image
import pickle
# import sys
# sys.setrecursionlimit(1000000000)

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing import image_dataset_from_directory
# from tfhub import load_model_weights

In [3]:
!unzip -qq /content/drive/Shareddrives/Hackathon/dataset-resized.zip -d /content/

In [4]:
!rm -r dataset-resized/trash/*
!rmdir dataset-resized/trash
!rmdir dataset-resized/.ipynb_checkpoints

rm: cannot remove 'dataset-resized/trash/*': No such file or directory
rmdir: failed to remove 'dataset-resized/trash': No such file or directory
rmdir: failed to remove 'dataset-resized/.ipynb_checkpoints': No such file or directory


In [5]:
os.listdir(os.path.join(os.getcwd(),"dataset-resized"))

['metal', 'organic', 'plastic', 'paper', 'glass', 'cardboard']

In [6]:
## get a path to the folder with images
path = os.path.join(os.getcwd(),"dataset-resized")
path

'/content/dataset-resized'

In [7]:
print(len(os.listdir('./dataset-resized//')))

6


In [8]:
train = tf.keras.preprocessing.image_dataset_from_directory(path, batch_size=16, image_size=(224,224), shuffle=True, label_mode='categorical', validation_split = 0.25, seed = 1, subset = 'training')
valid = tf.keras.preprocessing.image_dataset_from_directory(path, batch_size=16, image_size=(224,224), shuffle=True, label_mode='categorical', validation_split = 0.25, seed = 1, subset = 'validation')

Found 2909 files belonging to 6 classes.
Using 2182 files for training.
Found 2909 files belonging to 6 classes.
Using 727 files for validation.


In [9]:
model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3))

94773248/94765736 [==============================] - 1s 0us/step


In [10]:
model.trainable = False
model2 = tf.keras.Sequential()
model2.add(model)
model2.add(tf.keras.layers.Flatten())
# model2.add(tf.keras.layers.Dropout(0.2))
# model2.add(Dense(512, activation = 'relu'))
# model2.add(tf.keras.layers.Dropout(0.2))
# model2.add(Dense(256, activation = 'relu'))
# model2.add(tf.keras.layers.Dropout(0.2))
# model2.add(Dense(128, activation = 'relu'))
# model2.add(tf.keras.layers.Dropout(0.2))
# model2.add(Dense(64, activation = 'relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(6, activation='softmax'))

In [10]:
# model2.load_weights('/content/drive/MyDrive/waste segmentation/waste_weights.h5')

In [11]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dropout (Dropout)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 6)                 602118    
Total params: 24,189,830
Trainable params: 602,118
Non-trainable params: 23,587,712
_________________________________________________________________


In [12]:
model2.compile(optimizer=tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

In [13]:
model2.fit(x=train, validation_data=valid, epochs = 10)

Epoch 1/10
137/137 [==============================] - 44s 84ms/step - loss: 6.3781 - accuracy: 0.6647 - val_loss: 3.3979 - val_accuracy: 0.8459
Epoch 2/10
137/137 [==============================] - 10s 74ms/step - loss: 1.4336 - accuracy: 0.9269 - val_loss: 4.4661 - val_accuracy: 0.8514
Epoch 3/10
137/137 [==============================] - 10s 74ms/step - loss: 0.7554 - accuracy: 0.9583 - val_loss: 4.8072 - val_accuracy: 0.8336
Epoch 4/10
137/137 [==============================] - 10s 73ms/step - loss: 0.6589 - accuracy: 0.9671 - val_loss: 4.3122 - val_accuracy: 0.8652
Epoch 5/10
137/137 [==============================] - 10s 74ms/step - loss: 0.4967 - accuracy: 0.9783 - val_loss: 4.1543 - val_accuracy: 0.8762
Epoch 6/10
137/137 [==============================] - 10s 74ms/step - loss: 0.4797 - accuracy: 0.9762 - val_loss: 5.2997 - val_accuracy: 0.8693
Epoch 7/10
137/137 [==============================] - 10s 74ms/step - loss: 0.6037 - accuracy: 0.9819 - val_loss: 7.6798 - val_accuracy:

In [19]:
temp = img_to_array(load_img('/content/WhatsApp Image 2021-03-13 at 11.40.33 PM.jpeg', target_size=(224,224))).reshape(1,224,224,3)
np.argmax(model2.predict(temp))


3

In [17]:
model2.save_weights('model_weights_1.h5')

In [18]:
!cp model_weights_1.h5 /content/drive/Shareddrives/Hackathon/model_weights_1.h5